<!--
Copyright (c) 2025 Milin Patel
Hochschule Kempten - University of Applied Sciences

Autonomous Driving: AI Safety and Security Workshop
This project is licensed under the MIT License.
-->

*Copyright © 2025 Milin Patel. All Rights Reserved.*

# Notebook 23: ODD Definition and Runtime Monitoring

**Session 5: Standards Integration and Deployment**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/milinpatel07/Autonomous-Driving_AI-Safety-and-Security/blob/main/AV_Perception_Safety_Workshop/Session_5_Advanced_Topics/notebooks/23_ODD_Runtime_Monitoring.ipynb)

**Author:** Milin Patel

---

## Learning Objectives

- Understand what Operational Design Domain (ODD) is and why it's critical
- Learn to define ODD boundaries using ISO 34503 framework
- Implement runtime ODD monitoring systems
- Design graceful degradation strategies when leaving ODD
- Understand Minimal Risk Condition (MRC) and fallback mechanisms
- Analyze real-world ODD examples from deployed systems

---

In [ ]:
# Setup
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import seaborn as sns
from IPython.display import display, HTML

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✓ Setup complete")

---

## 1. What is ODD (Operational Design Domain)?

### Definition (SAE J3016)

**Operational Design Domain (ODD)**: A description of the specific operating domain(s) in which an automated driving system is designed to properly function.

### Why ODD Matters

**Key Principle**: 
- **No AV can operate everywhere** (Level 5 doesn't exist yet)
- Every system has **limitations**
- ODD **explicitly defines** these limitations

**Safety Argument**:
1. System is safe **within ODD**
2. System can **detect ODD exit**
3. System executes **safe fallback** when leaving ODD

### ODD vs. ASIL vs. SOTIF

| Concept | What It Defines | Example |
|---------|----------------|----------|
| **ODD** | Where/when system operates | "Highway only, daylight, dry weather" |
| **ASIL** | Required safety integrity | "Steering: ASIL D" |
| **SOTIF** | Performance limitations | "Cannot detect dark clothing at night" |

**Relationship**:
- ODD is designed to **avoid** SOTIF triggering conditions
- ASIL applies **within** the ODD
- Combined: "Within ODD, achieve ASIL X; outside ODD, execute MRC"

In [ ]:
# Visualization: ODD concept
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Chart 1: ODD as constraint space
from matplotlib.patches import Rectangle, Circle

ax1.set_xlim(0, 10)
ax1.set_ylim(0, 10)
ax1.set_aspect('equal')

# Full operational space
full_space = Rectangle((0, 0), 10, 10, fill=True, alpha=0.2, color='red', label='Full Operational Space')
ax1.add_patch(full_space)

# ODD (limited space)
odd_space = Rectangle((2, 2), 6, 5, fill=True, alpha=0.5, color='green', 
                       edgecolor='darkgreen', linewidth=3, label='Defined ODD')
ax1.add_patch(odd_space)

# Annotations
ax1.text(5, 9.3, 'All Possible Conditions', ha='center', fontsize=10, fontweight='bold')
ax1.text(5, 4.5, 'System Operates\nSafely Here', ha='center', va='center', 
         fontsize=11, fontweight='bold', color='darkgreen')
ax1.text(1, 8.5, 'Unsafe:\nSnow', fontsize=9, color='red')
ax1.text(8.5, 8.5, 'Unsafe:\nUnpaved', fontsize=9, color='red')
ax1.text(1, 1, 'Unsafe:\nNight', fontsize=9, color='red')
ax1.text(8.5, 1, 'Unsafe:\nConstruction', fontsize=9, color='red')

# ODD boundary markers
ax1.plot([2, 8], [7, 7], 'k--', linewidth=1.5, alpha=0.5)
ax1.plot([2, 8], [2, 2], 'k--', linewidth=1.5, alpha=0.5)
ax1.plot([2, 2], [2, 7], 'k--', linewidth=1.5, alpha=0.5)
ax1.plot([8, 8], [2, 7], 'k--', linewidth=1.5, alpha=0.5)

ax1.set_xlabel('Operational Conditions', fontsize=11, fontweight='bold')
ax1.set_ylabel('Environmental Conditions', fontsize=11, fontweight='bold')
ax1.set_title('ODD as Constrained Operating Space', fontsize=12, fontweight='bold')
ax1.legend(loc='upper left', fontsize=9)
ax1.set_xticks([])
ax1.set_yticks([])

# Chart 2: SAE Levels and typical ODD
levels = ['L0\nNo Auto', 'L1\nDriver\nAssist', 'L2\nPartial\nAuto', 'L3\nConditional\nAuto', 'L4\nHigh\nAuto', 'L5\nFull\nAuto']
odd_breadth = [0, 20, 40, 60, 75, 100]  # How broad the ODD typically is
colors = ['gray', 'lightblue', 'blue', 'orange', 'green', 'gold']

bars = ax2.barh(levels, odd_breadth, color=colors, alpha=0.7, edgecolor='black')
ax2.set_xlabel('Typical ODD Breadth (%)', fontsize=11, fontweight='bold')
ax2.set_title('SAE Automation Levels vs. ODD Scope', fontsize=12, fontweight='bold')
ax2.set_xlim(0, 110)
ax2.grid(True, alpha=0.3, axis='x')

# Annotations
annotations = [
    'No ODD (human drives)',
    'Very limited (cruise control)',
    'Limited (highway, good weather)',
    'Moderate (highway, most weather)',
    'Broad (city, most conditions)',
    'Unlimited (anywhere, anytime)\n[Does not exist yet!]'
]

for i, (bar, breadth, annotation) in enumerate(zip(bars, odd_breadth, annotations)):
    if breadth > 0:
        ax2.text(breadth + 3, bar.get_y() + bar.get_height()/2, 
                 annotation, va='center', fontsize=8, style='italic')

plt.tight_layout()
plt.show()

print("\n📍 Key Insight: ODD defines WHERE and WHEN the AV can operate safely")
print("   - Level 5 (unlimited ODD) does not exist yet")
print("   - Even Level 4 systems have significant ODD constraints")

---

## 2. ODD Attributes: ISO 34503 Framework

### ISO 34503: Road Vehicles - Test Scenarios for Automated Driving Systems

**Published**: 2023  
**Purpose**: Standardize ODD description and scenario-based testing

### Six Key ODD Attribute Categories

#### 1. **Roadway**
- **Type**: Highway, arterial, residential, parking lot, dirt road
- **Geometry**: Straight, curved, intersection, roundabout
- **Lanes**: Number, width, markings quality
- **Surface**: Paved, gravel, quality

#### 2. **Environment/Weather**
- **Precipitation**: None, light rain, heavy rain, snow, hail
- **Visibility**: Clear, fog, dust, smoke
- **Lighting**: Day, dusk, night, street lights
- **Temperature**: Operating range

#### 3. **Traffic**
- **Density**: Light, moderate, heavy, congestion
- **Types**: Cars, trucks, motorcycles, bicycles, pedestrians
- **Behavior**: Normal, aggressive, erratic

#### 4. **Speed**
- **Minimum**: E.g., 0 km/h (parking)
- **Maximum**: E.g., 60 km/h (UN R157 ALKS limit)
- **Typical**: Expected operating range

#### 5. **Geographic**
- **Region**: City, country, continent
- **HD Map**: Required or not
- **Geofencing**: Specific boundaries

#### 6. **Connectivity**
- **V2X**: Required or optional
- **Cloud**: Real-time updates needed?
- **Latency**: Maximum acceptable

### Example: Waymo Phoenix ODD

In [ ]:
# Waymo Phoenix ODD specification
waymo_odd = {
    'Attribute': ['Roadway Type', 'Road Geometry', 'Surface', 'Weather', 'Visibility', 
                  'Lighting', 'Speed Limit', 'Traffic Density', 'Geographic', 'HD Maps'],
    'Allowed Values': [
        'Urban streets, arterials (no highways, no unpaved)',
        'Grid layout, some curves, signalized intersections',
        'Paved, good condition only',
        'Dry, light rain only (no heavy rain, snow, hail)',
        'Clear to light fog only',
        'Day, dusk, night (with street lights)',
        '0-45 mph (0-72 km/h)',
        'Light to moderate (reduced service in heavy traffic)',
        '180 sq mi geofenced area in Phoenix metro',
        'Required (must be pre-mapped)'
    ],
    'Rationale': [
        'Phoenix grid layout is simpler than other cities',
        'Predictable road structure, well-maintained',
        'Lane markings clearly visible',
        'Phoenix has 300+ sunny days/year',
        'Dry climate, rare fog',
        'Good street lighting infrastructure',
        'Urban speed limits (safer if issues occur)',
        'Complexity increases with density',
        'Pre-mapped area only (cannot generalize)',
        'Critical for localization and path planning'
    ]
}

df_waymo = pd.DataFrame(waymo_odd)

print("\n" + "="*120)
print("WAYMO PHOENIX ODD SPECIFICATION (2024)")
print("="*120 + "\n")
display(df_waymo)

print("\n🚗 Key Insight: Even the most advanced AV has VERY constrained ODD")
print("   - Geofenced to 180 sq mi")
print("   - Dry/light rain only")
print("   - Must be pre-mapped")
print("   - Cannot handle unpaved roads, construction zones (without updates)")

---

## 3. Runtime ODD Monitoring

### Why Runtime Monitoring?

**Problem**: Conditions change while driving
- Sunny → sudden rainstorm
- Clear road → construction zone appears
- Low traffic → traffic jam

**Solution**: Continuously monitor whether current conditions are within ODD

### ODD Monitoring Architecture

```
┌─────────────────────────────────────────────────────────────┐
│                   SENSOR INPUTS                             │
│  - Weather sensors (rain, temperature)                     │
│  - Cameras (visibility, road condition)                    │
│  - LiDAR (range, point cloud density)                      │
│  - HD Map (localization confidence)                        │
│  - V2X (infrastructure status)                             │
└─────────────────────────────────────────────────────────────┘
                        ↓
┌─────────────────────────────────────────────────────────────┐
│               ODD ATTRIBUTE ESTIMATION                      │
│  - Weather classifier: dry/light rain/heavy rain           │
│  - Visibility estimator: >100m, 50-100m, <50m              │
│  - Road type detector: paved/unpaved                       │
│  - Traffic density: light/moderate/heavy                   │
└─────────────────────────────────────────────────────────────┘
                        ↓
┌─────────────────────────────────────────────────────────────┐
│                ODD COMPLIANCE CHECK                         │
│  For each attribute:                                        │
│    IF current_value IN allowed_values THEN OK              │
│    ELSE ODD_VIOLATION                                       │
└─────────────────────────────────────────────────────────────┘
                        ↓
┌─────────────────────────────────────────────────────────────┐
│              DECISION & FALLBACK                            │
│  - All attributes OK → Continue autonomous operation       │
│  - Minor violation → Graceful degradation                  │
│  - Major violation → Minimal Risk Condition (MRC)          │
└─────────────────────────────────────────────────────────────┘
```

### Implementation Example

In [ ]:
# ODD Monitor implementation
class ODDMonitor:
    """Runtime ODD compliance monitoring system"""
    
    def __init__(self, odd_spec):
        """
        Initialize ODD monitor with specification
        
        Args:
            odd_spec: Dictionary of ODD attributes and allowed values
        """
        self.odd_spec = odd_spec
        self.violation_history = []
        self.status = 'WITHIN_ODD'
    
    def check_attribute(self, attribute, current_value):
        """
        Check if current value is within ODD for given attribute
        
        Returns:
            (compliant: bool, severity: str)
        """
        allowed = self.odd_spec.get(attribute, {})
        
        if attribute == 'weather':
            if current_value in ['dry', 'light_rain']:
                return True, 'OK'
            elif current_value == 'moderate_rain':
                return False, 'MINOR'  # Can continue briefly
            else:  # heavy_rain, snow, hail
                return False, 'MAJOR'  # Must exit immediately
        
        elif attribute == 'visibility':
            if current_value >= 100:  # meters
                return True, 'OK'
            elif current_value >= 50:
                return False, 'MINOR'
            else:
                return False, 'MAJOR'
        
        elif attribute == 'speed':
            if 0 <= current_value <= allowed.get('max', 45):
                return True, 'OK'
            else:
                return False, 'MAJOR'
        
        elif attribute == 'road_type':
            if current_value in allowed.get('types', ['paved']):
                return True, 'OK'
            else:
                return False, 'MAJOR'
        
        elif attribute == 'map_confidence':
            if current_value >= 0.95:
                return True, 'OK'
            elif current_value >= 0.80:
                return False, 'MINOR'
            else:
                return False, 'MAJOR'
        
        # Default: if not explicitly allowed, it's a major violation
        return False, 'MAJOR'
    
    def monitor_step(self, current_conditions):
        """
        Monitor all ODD attributes for current timestep
        
        Args:
            current_conditions: Dict of current sensor readings
        
        Returns:
            (status, violations, recommended_action)
        """
        violations = []
        max_severity = 'OK'
        
        for attribute, value in current_conditions.items():
            compliant, severity = self.check_attribute(attribute, value)
            
            if not compliant:
                violations.append({
                    'attribute': attribute,
                    'current_value': value,
                    'severity': severity
                })
                
                # Track worst violation
                if severity == 'MAJOR' or (severity == 'MINOR' and max_severity == 'OK'):
                    max_severity = severity
        
        # Determine action
        if max_severity == 'OK':
            status = 'WITHIN_ODD'
            action = 'CONTINUE'
        elif max_severity == 'MINOR':
            status = 'ODD_BOUNDARY'
            action = 'GRACEFUL_DEGRADATION'  # Reduce speed, increase caution
        else:  # MAJOR
            status = 'OUTSIDE_ODD'
            action = 'MINIMAL_RISK_CONDITION'  # Pull over safely
        
        self.status = status
        self.violation_history.append({
            'timestamp': len(self.violation_history),
            'status': status,
            'violations': violations
        })
        
        return status, violations, action

# Example usage
odd_spec = {
    'weather': {'allowed': ['dry', 'light_rain']},
    'visibility': {'min': 100},  # meters
    'speed': {'max': 45},  # mph
    'road_type': {'types': ['paved']},
    'map_confidence': {'min': 0.95}
}

monitor = ODDMonitor(odd_spec)

# Simulate scenario: Driving in good conditions, then rain starts
scenarios = [
    {'name': 'Clear day', 'weather': 'dry', 'visibility': 200, 'speed': 35, 'road_type': 'paved', 'map_confidence': 0.98},
    {'name': 'Light rain starts', 'weather': 'light_rain', 'visibility': 150, 'speed': 30, 'road_type': 'paved', 'map_confidence': 0.97},
    {'name': 'Rain intensifies', 'weather': 'moderate_rain', 'visibility': 80, 'speed': 25, 'road_type': 'paved', 'map_confidence': 0.92},
    {'name': 'Heavy rain', 'weather': 'heavy_rain', 'visibility': 40, 'speed': 20, 'road_type': 'paved', 'map_confidence': 0.85},
    {'name': 'Enter construction', 'weather': 'moderate_rain', 'visibility': 60, 'speed': 15, 'road_type': 'unpaved', 'map_confidence': 0.65}
]

results = []
for scenario in scenarios:
    name = scenario.pop('name')
    status, violations, action = monitor.monitor_step(scenario)
    results.append({
        'Scenario': name,
        'Status': status,
        'Violations': len(violations),
        'Action': action
    })

df_results = pd.DataFrame(results)

print("\n" + "="*90)
print("ODD RUNTIME MONITORING SIMULATION")
print("="*90 + "\n")
display(df_results)

print("\n⚠️  Key Insight: System detects ODD violations in real-time")
print("   - Scenario 1-2: Normal operation")
print("   - Scenario 3: Minor violations → Graceful degradation (slow down)")
print("   - Scenario 4-5: Major violations → Minimal Risk Condition (pull over)")

---

## 4. Graceful Degradation and Minimal Risk Condition (MRC)

### Graceful Degradation

**Definition**: Reducing system capabilities while maintaining safe operation

**Strategies**:

| ODD Violation | Degradation Strategy |
|--------------|----------------------|
| Light rain detected | Reduce max speed by 20%, increase following distance |
| Visibility reduced | Slow to 50% max speed, activate all lights |
| Map confidence low | Switch to camera-only mode, reduce speed |
| Heavy traffic | Increase safety margins, more conservative lane changes |
| Minor sensor fault | Rely on redundant sensors, limit maneuvers |

**Goal**: Buy time to reach safe stopping location

### Minimal Risk Condition (MRC)

**Definition (ISO 21448)**: A condition to which the ADS transitions when a given trip cannot or should not be completed.

**MRC Strategies** (in order of preference):

#### 1. **Controlled Stop in Lane**
- Bring vehicle to stop in current lane
- Activate hazard lights
- **Use when**: Cannot safely exit lane (heavy traffic, no shoulder)

#### 2. **Pull Over to Shoulder**
- Change to rightmost lane
- Pull onto shoulder
- Stop, activate hazards
- **Use when**: Shoulder available, low traffic

#### 3. **Exit to Safe Location**
- Navigate to parking lot, gas station, etc.
- Park properly
- **Use when**: Violation anticipated (e.g., weather forecast), time available

#### 4. **Hand Over to Human**
- Alert driver/passenger
- Provide time to take control (e.g., 10 seconds)
- If no takeover: Execute MRC strategy 1-3
- **Use when**: Level 3 system only

### MRC Implementation Challenges

**Problem 1: Where to Stop?**
- Shoulder might not exist (urban)
- Stopping in lane creates new hazard
- Finding safe location requires perception to still work

**Problem 2: Communication**
- How to alert other drivers?
- How to call for assistance?
- Passengers might be asleep/incapacitated

**Problem 3: Secondary Crashes**
- Stopped AV might be hit by other vehicles
- Need to maximize visibility (hazards, lights, warning signals)

### Real-World Example: Waymo MRC

In [ ]:
# MRC decision tree
def determine_mrc_strategy(current_state):
    """
    Determine appropriate MRC strategy based on current conditions
    
    Args:
        current_state: Dict with road type, traffic, shoulder availability
    
    Returns:
        MRC strategy and estimated risk
    """
    road_type = current_state.get('road_type')
    traffic_density = current_state.get('traffic_density')
    shoulder_available = current_state.get('shoulder_available')
    speed = current_state.get('current_speed')
    time_available = current_state.get('time_to_violation')  # seconds
    
    # Decision logic
    if time_available > 60:  # More than 1 minute
        if road_type == 'urban':
            return 'EXIT_TO_PARKING', 'LOW'
        else:
            return 'PULL_TO_SHOULDER', 'LOW'
    
    elif time_available > 20:  # 20-60 seconds
        if shoulder_available and traffic_density in ['light', 'moderate']:
            return 'PULL_TO_SHOULDER', 'MODERATE'
        else:
            return 'SLOW_IN_LANE', 'MODERATE'
    
    else:  # Less than 20 seconds - emergency
        if shoulder_available:
            return 'EMERGENCY_SHOULDER', 'HIGH'
        else:
            return 'EMERGENCY_STOP_IN_LANE', 'VERY_HIGH'

# Test different scenarios
mrc_scenarios = [
    {'name': 'Highway, forecast rain', 'road_type': 'highway', 'traffic_density': 'light', 
     'shoulder_available': True, 'current_speed': 60, 'time_to_violation': 120},
    
    {'name': 'Urban, moderate traffic', 'road_type': 'urban', 'traffic_density': 'moderate', 
     'shoulder_available': False, 'current_speed': 30, 'time_to_violation': 45},
    
    {'name': 'Highway, sudden heavy rain', 'road_type': 'highway', 'traffic_density': 'heavy', 
     'shoulder_available': True, 'current_speed': 65, 'time_to_violation': 15},
    
    {'name': 'Urban, sensor failure', 'road_type': 'urban', 'traffic_density': 'heavy', 
     'shoulder_available': False, 'current_speed': 25, 'time_to_violation': 10},
    
    {'name': 'Highway, construction ahead', 'road_type': 'highway', 'traffic_density': 'moderate', 
     'shoulder_available': True, 'current_speed': 55, 'time_to_violation': 90}
]

mrc_results = []
for scenario in mrc_scenarios:
    name = scenario.pop('name')
    strategy, risk = determine_mrc_strategy(scenario)
    mrc_results.append({
        'Scenario': name,
        'Road Type': scenario['road_type'],
        'Traffic': scenario['traffic_density'],
        'Time Available': f"{scenario['time_to_violation']}s",
        'MRC Strategy': strategy,
        'Risk Level': risk
    })

df_mrc = pd.DataFrame(mrc_results)

print("\n" + "="*110)
print("MRC STRATEGY SELECTION")
print("="*110 + "\n")
display(df_mrc)

print("\n🛑 Key Insight: MRC strategy depends on available time and road conditions")
print("   - Proactive (>60s): Navigate to safe location")
print("   - Reactive (20-60s): Pull to shoulder if possible")
print("   - Emergency (<20s): Stop as safely as possible")
print("   - ALL MRC maneuvers carry risk - prevention is better!")

---

## 5. Driver Handover (Level 3 Only)

### SAE Level 3 Challenge

**Level 3 Definition**: ADS performs all driving, but human must be ready to take over when requested

**The Handover Problem**:
- Driver might be:
  - Reading
  - Sleeping
  - Watching video
  - Not paying attention
- **How much time needed to safely resume control?**

### Research Findings

**Studies (Gold et al., 2016; Eriksson & Stanton, 2017)**:
- Minimum takeover time: **5-7 seconds**
- Average takeover time: **10-15 seconds**
- During takeover: **Degraded driving performance for 20-40 seconds**

**Implication**: Level 3 requires:
1. Predict ODD exit **at least 15 seconds in advance**
2. Alert driver with escalating warnings
3. If no takeover: Execute MRC

### UN R157 ALKS Requirements

**Handover Procedure**:
1. **Initial warning**: Visual + audible, 10 seconds before handover needed
2. **Escalation**: Haptic (seat vibration) if no response after 3 seconds
3. **Final warning**: Loud audible, flashing lights, 7 seconds before MRC
4. **No takeover**: Execute MRC (stop vehicle)

### Why Level 4 Avoids This Problem

**Level 4**: No human takeover required
- System must handle ALL situations within ODD
- When ODD exit: Execute MRC autonomously
- No reliance on driver readiness

**Trade-off**: Level 4 requires more capable system but safer (no handover uncertainty)

In [ ]:
# Handover timeline visualization
fig, ax = plt.subplots(figsize=(14, 6))

# Timeline events
events = [
    {'time': 0, 'event': 'ODD Exit Detected', 'color': 'orange'},
    {'time': 5, 'event': 'Initial Warning\n(Visual + Audible)', 'color': 'yellow'},
    {'time': 8, 'event': 'Escalated Warning\n(Haptic)', 'color': 'orange'},
    {'time': 12, 'event': 'Final Warning\n(Loud + Flashing)', 'color': 'red'},
    {'time': 15, 'event': 'MRC Initiated\n(No Takeover)', 'color': 'darkred'},
    {'time': 25, 'event': 'Vehicle Stopped', 'color': 'black'}
]

# Plot timeline
for event in events:
    ax.axvline(x=event['time'], color=event['color'], linewidth=3, alpha=0.7)
    ax.text(event['time'], 1.5, event['event'], rotation=0, va='bottom', ha='center',
            fontsize=10, fontweight='bold', 
            bbox=dict(boxstyle='round', facecolor=event['color'], alpha=0.3))

# Driver response zones
ax.axhspan(0.2, 0.6, xmin=0, xmax=5/30, alpha=0.3, color='green', label='Autonomous Operation')
ax.axhspan(0.2, 0.6, xmin=5/30, xmax=15/30, alpha=0.3, color='yellow', label='Handover Window')
ax.axhspan(0.2, 0.6, xmin=15/30, xmax=25/30, alpha=0.3, color='red', label='MRC Execution')

# Typical driver response time
ax.axvspan(8, 13, alpha=0.2, color='blue', label='Typical Driver Takeover Time')

ax.set_xlim(-1, 30)
ax.set_ylim(0, 2)
ax.set_xlabel('Time (seconds)', fontsize=12, fontweight='bold')
ax.set_title('Level 3 Handover Procedure Timeline (UN R157 ALKS)', fontsize=14, fontweight='bold')
ax.set_yticks([])
ax.legend(loc='upper right', fontsize=10)
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("\n⏱️  Key Insight: Handover takes 10-15 seconds on average")
print("   - System must predict ODD exit well in advance")
print("   - Multiple escalating warnings required")
print("   - If driver doesn't respond: MRC is safety net")
print("   - Level 4 eliminates handover uncertainty (safer)")

---

## 6. Real-World ODD Examples

### Comparison of Deployed Systems

In [ ]:
# Comparison of ODD for different systems
odd_comparison = {
    'System': ['Waymo Phoenix', 'Cruise SF (before shutdown)', 'Mercedes Drive Pilot', 'Tesla Autopilot', 'UN R157 ALKS'],
    'SAE Level': ['4', '4', '3', '2', '3'],
    'Geography': ['180 sq mi Phoenix', '45 sq mi SF', 'Germany highways', 'Global', 'Highway only'],
    'Speed': ['0-45 mph', '0-35 mph', '0-60 km/h', '0-85 mph', '0-60 km/h'],
    'Weather': ['Dry, light rain', 'Dry, light rain', 'Dry only', 'All weather', 'Dry, light rain'],
    'Roads': ['Urban, paved', 'Urban, paved', 'Highway, paved', 'All paved', 'Highway, paved'],
    'Lighting': ['Day, night (lights)', 'Day, night (lights)', 'Day only', 'All', 'Day, dusk'],
    'HD Maps': ['Required', 'Required', 'Required', 'Not required', 'Recommended'],
    'Handover': ['No (L4)', 'No (L4)', 'Yes (10s)', 'Immediate (L2)', 'Yes (10s)']
}

df_comparison = pd.DataFrame(odd_comparison)

print("\n" + "="*140)
print("ODD COMPARISON: DEPLOYED AV SYSTEMS (2024)")
print("="*140 + "\n")
display(df_comparison)

print("\n📊 Key Insights:")
print("   - Level 4 systems: Very narrow ODD (geofenced, weather-limited)")
print("   - Level 3 systems: Moderate ODD but require driver handover")
print("   - Level 2 (Tesla): Broad ODD but constant supervision required")
print("   - Trade-off: Broader ODD = lower automation level")

---

## 7. Summary and Best Practices

### Key Takeaways

1. **ODD is Mandatory**
   - No AV can operate everywhere (Level 5 doesn't exist)
   - ODD explicitly defines system limitations
   - Clear ODD = safer deployment

2. **ODD Must Be**:
   - **Specific**: "Good weather" is not enough; define "dry or light rain <5mm/hr"
   - **Verifiable**: Must be able to detect when ODD is violated
   - **Conservative**: Better to have narrow ODD and be safe than broad ODD and fail

3. **Runtime Monitoring is Critical**
   - Conditions change while driving
   - Must continuously check all ODD attributes
   - Need multiple redundant sensors for ODD monitoring

4. **Graceful Degradation > Emergency Stop**
   - Predict ODD violations in advance when possible
   - Use graceful degradation to buy time
   - MRC is last resort (all MRC maneuvers carry risk)

5. **Level 3 Handover is Problematic**
   - Requires 10-15 seconds for safe takeover
   - Driver might not respond
   - Level 4 (no handover) is safer architecture

6. **ODD Defines Deployment Strategy**
   - Narrow ODD → Geofenced deployment (Waymo)
   - Moderate ODD → Highway only (ALKS)
   - Broad ODD → Lower automation level (Tesla L2)

### Best Practices for ODD Design

#### 1. Start Narrow, Expand Gradually
- Begin with very constrained ODD (e.g., one neighborhood, daylight, dry)
- Validate thoroughly
- Expand incrementally (add rain, add night, etc.)

#### 2. Use ISO 34503 Framework
- Cover all 6 attribute categories
- Document allowed values and rationale
- Make ODD machine-readable for runtime monitoring

#### 3. Design for Graceful Exits
- Always know where nearest safe stopping location is
- Plan routes that allow for ODD exits
- Monitor weather forecasts and road construction databases

#### 4. Test ODD Boundaries
- Don't just test within ODD
- Test ODD violation detection
- Test MRC execution in various scenarios
- Verify handover procedures (if Level 3)

#### 5. Communicate ODD to Users
- Users must understand limitations
- In-vehicle displays: "System will exit ODD in 2 minutes (heavy rain forecast)"
- App notifications before trip: "Route includes areas outside ODD"

### Future Directions

**Adaptive ODD**:
- Current: Static ODD (same for all vehicles)
- Future: Adaptive ODD based on vehicle health, sensor status
  - "Normally operate in light rain, but if LiDAR degraded, restrict to dry only"

**Predictive ODD Monitoring**:
- Use weather forecasts, traffic predictions
- Plan routes that stay within ODD
- Proactively exit ODD before conditions deteriorate

**Cooperative ODD**:
- Share ODD status between vehicles (V2V)
- "Heavy rain 2 miles ahead, recommend ODD exit"

**Standardized ODD Taxonomy**:
- Need industry-wide standard for describing ODD
- Allow regulators to compare systems objectively

In [ ]:
# Final summary visualization
fig = plt.figure(figsize=(14, 10))
gs = fig.add_gridspec(2, 2, hspace=0.3, wspace=0.3)

# Chart 1: ODD Breadth vs Safety
ax1 = fig.add_subplot(gs[0, 0])
odd_breadth = np.array([20, 40, 60, 80, 100])
safety_confidence = 100 - (odd_breadth * 0.7)  # Inverse relationship

ax1.plot(odd_breadth, safety_confidence, 'r-', linewidth=3, marker='o', markersize=8)
ax1.fill_between(odd_breadth, safety_confidence, alpha=0.3, color='red')
ax1.set_xlabel('ODD Breadth (%)', fontsize=11, fontweight='bold')
ax1.set_ylabel('Safety Confidence (%)', fontsize=11, fontweight='bold')
ax1.set_title('Trade-off: ODD Breadth vs. Safety Confidence', fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.set_xlim(10, 110)
ax1.set_ylim(0, 110)

# Annotations
ax1.annotate('Waymo\n(narrow ODD)', xy=(25, 82), xytext=(15, 95),
             arrowprops=dict(arrowstyle='->', color='green', lw=1.5),
             fontsize=9, fontweight='bold')
ax1.annotate('Tesla L2\n(broad ODD,\nlow automation)', xy=(85, 40), xytext=(70, 20),
             arrowprops=dict(arrowstyle='->', color='blue', lw=1.5),
             fontsize=9, fontweight='bold')

# Chart 2: ODD attribute coverage
ax2 = fig.add_subplot(gs[0, 1])
attributes = ['Weather', 'Lighting', 'Roads', 'Speed', 'Geography', 'Connectivity']
waymo_coverage = [30, 60, 40, 50, 5, 80]
level5_target = [100, 100, 100, 100, 100, 100]

x = np.arange(len(attributes))
width = 0.35

ax2.bar(x - width/2, waymo_coverage, width, label='Waymo (L4)', color='green', alpha=0.7)
ax2.bar(x + width/2, level5_target, width, label='Level 5 Target', color='gold', alpha=0.7)

ax2.set_ylabel('ODD Coverage (%)', fontsize=11, fontweight='bold')
ax2.set_title('ODD Attribute Coverage: Current vs. Target', fontsize=12, fontweight='bold')
ax2.set_xticks(x)
ax2.set_xticklabels(attributes, rotation=45, ha='right')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3, axis='y')
ax2.set_ylim(0, 110)

# Chart 3: MRC strategies by frequency
ax3 = fig.add_subplot(gs[1, 0])
mrc_types = ['Pull to\nShoulder', 'Exit to\nParking', 'Stop in\nLane', 'Emergency\nStop']
usage_percent = [45, 35, 15, 5]
risk_level = [30, 20, 60, 90]

colors_mrc = ['green', 'yellow', 'orange', 'red']
bars = ax3.barh(mrc_types, usage_percent, color=colors_mrc, alpha=0.7, edgecolor='black')

ax3.set_xlabel('Usage Frequency (%)', fontsize=11, fontweight='bold')
ax3.set_title('MRC Strategies: Usage vs. Risk', fontsize=12, fontweight='bold')
ax3.grid(True, alpha=0.3, axis='x')

for i, (bar, usage, risk) in enumerate(zip(bars, usage_percent, risk_level)):
    ax3.text(usage + 2, bar.get_y() + bar.get_height()/2, 
             f'{usage}% (Risk: {risk})',
             va='center', fontsize=9, fontweight='bold')

# Chart 4: ODD monitoring architecture
ax4 = fig.add_subplot(gs[1, 1])
ax4.axis('off')

# Text summary
summary_text = """
ODD MONITORING BEST PRACTICES

✓ Define ODD precisely (ISO 34503)
✓ Monitor all attributes in real-time
✓ Use redundant sensors
✓ Predict violations when possible
✓ Graceful degradation before MRC
✓ Test ODD boundary conditions
✓ Communicate ODD to users
✓ Start narrow, expand gradually

KEY METRICS:
• ODD violation detection: >99.9%
• False alarm rate: <0.1%
• MRC success rate: >99.99%
• Graceful degradation: 80%+ of exits
"""

ax4.text(0.1, 0.9, summary_text, transform=ax4.transAxes,
         fontsize=10, verticalalignment='top', family='monospace',
         bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

plt.suptitle('ODD Runtime Monitoring: Summary', fontsize=16, fontweight='bold', y=0.98)

plt.show()

print("\n" + "="*100)
print("NOTEBOOK COMPLETE: ODD DEFINITION AND RUNTIME MONITORING")
print("="*100)
print("\n🎯 Main Lessons:")
print("   1. ODD is mandatory - defines where/when AV can operate safely")
print("   2. Runtime monitoring ensures continuous ODD compliance")
print("   3. Graceful degradation > emergency stops")
print("   4. MRC is safety net, but all MRC maneuvers carry risk")
print("   5. Narrow ODD = higher safety confidence")
print("   6. Level 5 (unlimited ODD) does not exist yet")
print("\n📚 Next: Notebook 24 - Standards Gaps and Open Problems")
print("="*100)

---

## References

### Standards
- **ISO 34503:2023** - Road vehicles — Test scenarios for automated driving systems — Vocabulary and data formats
- **SAE J3016:2021** - Taxonomy and Definitions for Terms Related to Driving Automation Systems for On-Road Motor Vehicles
- **UN R157:2021** - Automated Lane Keeping Systems (ALKS)
- **ISO 21448:2022** - SOTIF (includes MRC requirements)

### Research Papers
- Gold, C., et al. (2016). "Taking Over Control From Highly Automated Vehicles" *Human Factors*, 58(4), 642-652.
- Eriksson, A., & Stanton, N. A. (2017). "Takeover Time in Highly Automated Vehicles" *Human Factors*, 59(4), 689-705.
- Koopman, P., & Fratrik, F. (2019). "How Many Operational Design Domains, Objects, and Events?" *SAE Technical Paper 2019-01-0317*.

### Industry Reports
- Waymo Safety Report (2024)
- NHTSA ODD Framework (2023)

---

*Copyright © 2025 Milin Patel. All Rights Reserved.*